In [1]:
import pandas as pd
import seaborn as sbs
import numpy as np
import calendar
from sklearn.metrics import mean_squared_error
from sklearn import linear_model,tree
from sklearn.model_selection import train_test_split
from sklearn import neural_network
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from math import sqrt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [2]:
subte = pd.read_csv("data/estaciones-de-subte.csv")
properati = pd.read_csv("data/properati_full_month.csv",low_memory=False)
test = pd.read_csv("data/properati_dataset_testing_noprice.csv")

# Dropeo de columnas, asi quedan misma cantidad de columnas en el train ( properati ) y en el test, solo me quedo con price_aprox_usd

In [3]:
properati.drop(inplace=True,labels=\
               ['image_thumbnail','title','surface_in_m2','price_aprox_local_currency','price',\
               'price_per_m2','price_usd_per_m2','properati_url','title','geonames_id','currency','extra'],axis=1)

In [4]:
properati.country_name = properati.place_with_parent_names.apply(lambda x : x.split('|')[1])
properati = properati[properati['country_name'] == 'Argentina']
states = properati.place_with_parent_names.apply(lambda x : x.split('|')[2])
testStates = test.place_with_parent_names.apply(lambda x: x.split('|')[2])

In [5]:
properati.loc[:,'state_name'] = states
test.loc[:,'state_name'] = testStates

In [6]:
properati = properati[
    (properati['state_name'] == 'Bs.As. G.B.A. Zona Norte' ) |
    (properati['state_name'] == 'Bs.As. G.B.A. Zona Sur' ) |
    (properati['state_name'] == 'Bs.As. G.B.A. Zona Oeste' ) |
    (properati['state_name'] == 'Capital Federal' ) 
    ]

In [7]:
properati['zone'] = properati.place_with_parent_names.apply(lambda x : x.split('|')[3])
test['zone'] = test.place_with_parent_names.apply(lambda x : x.split('|')[3])

# Relleno los que en zone no tienen nada, los relleno con el place_name

In [8]:
properati.loc[properati['zone'] == '','zone'] = properati.loc[properati['zone'] == '','place_name']

In [9]:
test.loc[test['zone'] == '','zone'] = test.loc[test['zone'] == '','place_name']

In [10]:
# Filtro las stores
properati = properati.loc[properati['property_type'] != 'store',:]

In [11]:
properati['property_type'] = properati['property_type'].map({'apartment':0, 'house':1, 'PH' :2})
test['property_type'] = test['property_type'].map({'departamento':0, 'casa':1, 'ph' :2})

In [12]:
train = properati.copy()

In [13]:
test['state'] = test['state_name'].map({'Capital Federal':0, 'Bs.As. G.B.A. Zona Norte':1, 'Bs.As. G.B.A. Zona Oeste' :2,\
                                       'Bs.As. G.B.A. Zona Sur': 3,'Buenos Aires Interior': 4})
train['state'] = train['state_name'].map({'Capital Federal':0, 'Bs.As. G.B.A. Zona Norte':1, 'Bs.As. G.B.A. Zona Oeste' :2,\
                                       'Bs.As. G.B.A. Zona Sur': 3})

### Saco la cercania a subtes

In [14]:
def subtes(x):
    # Veo si esta null o no
    if ( (x.lat) and (x.lon) ):
        # si esta en capital
        if ( x.state_name == 'Capital Federal'):
            # defino el rango a buscar
            rango = 0.0044
            latitud = x.lat
            longitud = x.lon
            coinciden = subte.loc[ (subte['Y'].between(latitud - rango, latitud + rango,inclusive=True)) & 
                 (subte['X'].between(longitud - rango, longitud + rango, inclusive=True) ),:]
            # si hay una linea cerca la devuelve
            if ( not(coinciden.empty) ):
                return coinciden['LINEA'].iloc[0]
            else:
                return 'N'
    # si no encuentra ninguno de los otros casos sale con N         
    return 'NULL'

In [15]:
train = train.loc[pd.notnull(train['price_aprox_usd'])]

In [16]:
subtesTest = test.apply(subtes,axis=1)

In [17]:
subtes = train.apply(subtes,axis=1)

In [18]:
train['subte'] = subtes
test['subte'] = subtesTest

In [19]:
test['subte'] = test['subte'].map({'NULL':0, 'N':1, 'A' :2, 'B' : 3, 'C': 4, 'D' : 5, 'E' : 6, 'H': 7})
train['subte'] = train['subte'].map({'NULL':0, 'N':1, 'A' :2, 'B' : 3, 'C': 4, 'D' : 5, 'E' : 6, 'H': 7})

In [20]:
trainToTest = train.copy()

In [21]:
toTest = test.copy()

In [22]:
trainToTest = train[['property_type','zone','state','subte']]
toTest = test[['property_type','zone','state','subte']]

# ONE HOT ENCODING

In [23]:
one_hot_encoding = pd.get_dummies(trainToTest['zone'])
trainToTest = trainToTest.drop('zone',axis=1)
trainToTest = trainToTest.join(one_hot_encoding)
trainToTest

,property_type,state,subte,Abasto,Agronomía,Almagro,Almirante Brown,Avellaneda,Balvanera,Barracas,...,Villa Luro,Villa Ortuzar,Villa Pueyrredón,Villa Real,Villa Riachuelo,Villa Santa Rita,Villa Soldati,Villa Urquiza,Villa del Parque,Zona Portuaria
0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0,0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0,0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0,0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
one_hot_encoding = pd.get_dummies(toTest['zone'])
toTest = toTest.drop('zone',axis=1)
toTest = toTest.join(one_hot_encoding)
toTest

,property_type,state,subte,Abasto,Agronomía,Almagro,Almirante Brown,Avellaneda,Balvanera,Barracas,...,Villa Lugano,Villa Luro,Villa Ortuzar,Villa Pueyrredón,Villa Real,Villa Riachuelo,Villa Santa Rita,Villa Soldati,Villa Urquiza,Villa del Parque
0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0,0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
trainToTest = trainToTest.drop(['Malvinas Argentinas','Catalinas','San Isidro','Bs.As. G.B.A. Zona Norte', \
                                 'Distrito de las Artes','Distrito Audiovisual','José C Paz','San Fernando',\
                                 'Tigre','Zona Portuaria','Pilar','Escobar','Distrito Tecnológico','San Miguel'],axis=1)

In [26]:
toTest = toTest.drop(['Buenos Aires Interior'],axis=1)

In [27]:
train_price = train['price_aprox_usd']

In [28]:
trainToTest['surface_covered_in_m2'] = train['surface_covered_in_m2']
toTest['surface_covered_in_m2'] = test['surface_covered_in_m2']
trainToTest['rooms'] = train['rooms']
toTest['rooms'] = test['rooms']

In [29]:
trainToTest = trainToTest.fillna(0)
toTest = toTest.fillna(0)

In [30]:
train_price.shape

(390061L,)

In [31]:
trainToTest.shape

(390061, 88)

In [32]:
toTest.shape

(14166, 88)

In [33]:
#Random Forest Regression
rfr = RandomForestRegressor(n_estimators = 200, min_samples_leaf = 4, random_state = 0)

x1_train, x1_test, y1_train, y1_test = train_test_split(trainToTest, train_price, test_size = 0.4, random_state = 0)

rfr.fit(x1_train,y1_train)

prediction_rfr200 = rfr.predict(x1_test)
rms = sqrt(mean_squared_error(prediction_rfr200, y1_test))
print "Random Forest aplicado"
rms

Random Forest aplicado


238369.95401286456

In [ ]:
# #gradient boosting regresion
# gbr = GradientBoostingRegressor(n_estimators=400,min_samples_leaf=3)
# x2_train, x2_test, y2_train, y2_test = train_test_split(trainToTest, train_price, test_size = 0.4, random_state = 0)
# gbr.fit(x2_train,y2_train)
# #prediccin y score
# prediction_gbrRoom = gbr.predict(x2_test)
# rms = sqrt(mean_squared_error(prediction_gbrRoom, y2_test))
# rms

In [ ]:
# prediction_avg = []
# for pos in range(0, len(prediction_gbrRoom)):
    
#     avg = float( (prediction_gbrRoom[pos] + prediction_rfr200[pos] ) /2 )
#     prediction_avg.append(avg)

In [ ]:
# rms = sqrt(mean_squared_error(prediction_avg, y2_test))
# rms

In [34]:
#Random Forest Regression
rfr = RandomForestRegressor(n_estimators = 200, min_samples_leaf = 4, random_state = 0)
rfr.fit(trainToTest,train_price)

prediction_rfrTestRoom = rfr.predict(toTest)

In [ ]:
# #gradient boosting regresion
# gbr = GradientBoostingRegressor(n_estimators=1000,min_samples_leaf=4)
# gbr.fit(trainToTest,train_price)
# prediction_gbrTestRoom = gbr.predict(toTest)

# Ensamble

In [ ]:
# prediction_avgTest = []
# for pos in range(0, len(prediction_gbrTestRoom)):
    
#     avg = float( (prediction_gbrTestRoom[pos] + prediction_rfrTestRoom[pos] ) /2 )
#     prediction_avgTest.append(avg)

In [ ]:
# test['price_usd'] = prediction_avgTest

In [ ]:
# testASubir = test[['id','price_usd']]

In [ ]:
# testASubir.to_csv('data/SubmitEnsambleRFRyGBRFinal.csv',index=False)

In [35]:
test['price_usd'] = prediction_rfrTestRoom
testASubir = test[['id','price_usd']]
testASubir.to_csv('data/SubmitRFRFinal.csv',index=False)